In [1]:
from lxml import html
import requests
import re
import json
from dataclasses import dataclass, asdict

In [2]:
@dataclass
class Course:
    title: str
    course_type: str

In [3]:
# Params
base_url = "https://didattica.unipd.it"
courses_types = ["LT", "LM", "CU"]
years = range(2012,2022)
urls = {}
for t in courses_types:
    urls[t] = [];
courses = []

In [4]:
# Download the list of schools
for t in courses_types:
    for y in years:
        page = requests.get( base_url + "/off/" + str(y) + "/" + t )
        tree = html.fromstring( page.content )
        urls[t].extend( tree.xpath('//tr[@class="list_scuole"]//a/@href') )

In [ ]:
# Download the list of courses
courses_dict = {}
for t in courses_types:
    for u in urls[t]:
        page = requests.get( base_url + u )
        tree = html.fromstring( page.content )
        rows = tree.xpath('//tr[@class="list_cds"]//tbody//tr')
        for r in rows:
            cells = r.xpath('td')
            if( len(cells) ):
                course_title = cells[1].xpath('a//text()')
                if( isinstance( course_title, list) ):
                    course_title = course_title[0]
                course_title = course_title.title()
                course_title = re.sub(r'\(.*\)',"",course_title)
                course_title = course_title.capitalize()
                if course_title in courses_dict:
                    continue
                else:
                    courses_dict[course_title] = Course( course_title, t )
courses_dict["Scienze Morali"] = Course( "Scienze Morali", "SG")
courses_dict["Scienze Naturali"] = Course( "Scienze Naturali", "SG")
courses_dict["Scienze Sociali"] = Course( "Scienze Sociali", "SG")

In [ ]:
all_courses = list( courses_dict.values() )
all_courses.sort( key= lambda x : x.title )
all_courses = list( map( asdict, all_courses ) )
with open('courses.json', 'w') as outfile:
    json.dump( all_courses, outfile )